In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
# 16 Oct 2023: gdal must be imported first, and then rasterio can be imported
from osgeo import gdal
import rasterio as rio

In [2]:
import rasterio2ENVIhdr

In [3]:
# Set some directories. Here we use absolute directories. 
cwd = 'c:\\Users\\m1865\\Desktop\\Ticino'
cwd_Field = cwd + '\\FieldData'
cwd_PRISMA = cwd + "\\PRISMA Raster Raw\\Merged"
cwd_PRISMA_Mask = cwd + "\\PRISMA Raster Mask"
cwd_PRISMA_Indices = cwd + "\\PRISMA Classification Indices"

In [4]:
# Get the name of all the rasters in our folder
raster_Names = [name.split('.')[0] for name in os.listdir(cwd_PRISMA)]
raster_Names = list(set(raster_Names))
raster_Names

['PRS_L2D_STD_20220611_20220710_NS_mosaic_crop_smooth_v2i',
 'PRS_L2D_STD_20220906_20220911_NS_mosaic_crop_smooth_v2i']

In [5]:
raster_Paths = []
for name in raster_Names:
    temp = cwd_PRISMA + "\\" + name
    raster_Paths.append(temp)
raster_Paths

['c:\\Users\\m1865\\Desktop\\Ticino\\PRISMA Raster Raw\\Merged\\PRS_L2D_STD_20220611_20220710_NS_mosaic_crop_smooth_v2i',
 'c:\\Users\\m1865\\Desktop\\Ticino\\PRISMA Raster Raw\\Merged\\PRS_L2D_STD_20220906_20220911_NS_mosaic_crop_smooth_v2i']

In [6]:
raster_Raw = []
for raster in raster_Paths:
    temp = rio.open(raster)
    raster_Raw.append(temp)
raster_Raw

[<open DatasetReader name='c:/Users/m1865/Desktop/Ticino/PRISMA Raster Raw/Merged/PRS_L2D_STD_20220611_20220710_NS_mosaic_crop_smooth_v2i' mode='r'>,
 <open DatasetReader name='c:/Users/m1865/Desktop/Ticino/PRISMA Raster Raw/Merged/PRS_L2D_STD_20220906_20220911_NS_mosaic_crop_smooth_v2i' mode='r'>]

## Import our extended shapefile

In [7]:
gdf = gpd.read_file(cwd_Field + "\\ShapefileCorrettiAggiunto\\confini_foreste_estesi.shp")

## Crop our raster with the shapefile! 

In [8]:
# There are some tags exclusive to ENVI header files, but "rasterio" fails to embed them. Thus we need to extract them and later create the correct ENVI header file by ourselves. 
# Since all the raster files have the same info regarding band names, wavelength and wavelength units, here we extract them from the first raster. 
str_ENVI_hdr_band_names = 'band names = ' + raster_Raw[0].tags(ns='ENVI')['band_names']
str_ENVI_hdr_wavelength = 'wavelength = ' + raster_Raw[0].tags(ns='ENVI')['wavelength']
str_ENVI_hdr_wavelength_units = 'wavelength units = ' + raster_Raw[0].tags(ns='ENVI')['wavelength_units']

In [9]:
# Clip the rasters and save them to local storage! 
import rasterio.mask
count = 0
for raster in raster_Raw: 
    out_image, out_transform = rasterio.mask.mask(raster,gdf.geometry,crop=True)
    out_meta = raster.meta
    out_meta.update({"driver": 'ENVI',
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform,                    
                    })
    with rasterio.open(cwd_PRISMA_Indices + '\\' + raster_Names[count] + ' Cropped', "w", **out_meta) as dest:
        dest.write(out_image)
    print(raster_Names[count] + 'Cropped has been saved successfully! ')
    count = count + 1

PRS_L2D_STD_20220611_20220710_NS_mosaic_crop_smooth_v2iCropped has been saved successfully! 
PRS_L2D_STD_20220906_20220911_NS_mosaic_crop_smooth_v2iCropped has been saved successfully! 
